In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import tifffile
import sys, time, os, csv
import cProfile, pstats
from jutils import tqdm_alias as tqdm

import psfmatrix, lfimage
import projector, lfdeconv
import special_fftconvolve as special
import jutils as util
import py_light_field as plf

In [ ]:
projectorClass = projector.Projector_allC
maxiter = 8

### Do the actual deconvolution of some of Nils's data

Note that, compared to Prevedel's PSF generation code, I have fixed the max bug, and also flux-normalised H.

In [ ]:
hMatrix = psfmatrix.LoadMatrix('PSFmatrix/fdnormPSFMatrix_M22.2NA0.5MLPitch125fml3125from-156to156zspacing4Nnum19lambda520n1.33.mat')
inputImage = []
lr = 'Right'
for i in range(40, 41):
    im = lfimage.LoadLightFieldTiff('/Users/jonny/Movies/Nils files/Rectified/%s/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39))
    inputImage.append(im)
inputImage = np.array(inputImage)
print(inputImage.shape)

In [ ]:
#plf.SetThreadFileName("threads.txt")
t1 = time.time()
Htf = lfdeconv.BackwardProjectACC(hMatrix, inputImage, planes=None, progress=tqdm, logPrint=False, projector=projectorClass())
t2 = time.time()
#plf.SetThreadFileName("")
print(t2-t1)
print(Htf.shape)

In [ ]:
deconvolvedResult2 = lfdeconv.DeconvRL(hMatrix, Htf, maxIter=2, Xguess=Htf.copy(), logPrint=False, projector=projectorClass())

In [ ]:
deconvolvedResult = lfdeconv.DeconvRL(hMatrix, Htf, maxIter=maxiter, Xguess=Htf.copy(), logPrint=False, projector=projectorClass())
print(deconvolvedResult.shape)
# For "Right", this is now taking 770s per iteration. elapsed wallclock time 6189.126849, rusage 47236.057507
# It should be faster than that (I thought left previously took more like 600 or possibly 660 - for 8 frames I think)
# What is going on?
# 975s when processing 11 frames
# OK, after the bug fix it's down to 450s/iter, giving these totals for 8 timepoints:
# Deconvolution elapsed wallclock time 3631.499198, rusage 26969.383617
# For 11 timepoints it takes about 565s/iter:
# Deconvolution elapsed wallclock time 4595.855735, rusage 33892.436672

In [ ]:
plt.imshow(np.max(deconvolvedResult, axis=0)[0])
plt.show()
plt.imshow(np.max(deconvolvedResult2, axis=0)[0])
plt.show()

In [ ]:
if True:
    # Save the results
    for i in range(deconvolvedResult.shape[1]):
        tifffile.imsave('Data/03_Reconstructed/%s/Cam_%s_%d_X%d_N19.tif'%(lr, lr, 40+i, 1+i), 
                        deconvolvedResult[:,i].transpose((0,2,1)))# Transpose to match Matlab image appearance

In [ ]:
if False:
    # Calculate MIPs, for inspection purposes
    for i in range(40, 51):
        im = tifffile.imread('Data/03_Reconstructed/%s/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39))
        tifffile.imsave('Data/03_Reconstructed/z-mips-%s/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39), 
                        np.max(im, axis=0))
        tifffile.imsave('Data/03_Reconstructed/y-mips-%s/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39), 
                        np.max(im, axis=1))

### Create partial camera images by subtracting contributions of outer planes
Forward-project contribution from planes *other* than planes 25-53 (inclusive), and subtract that from the raw camera images

In [ ]:
zCropSlice = slice(25,54)
# Reload image, to save having to recalculate it from scratch(!),
# and mask OUT the planes we are ultimately interested in
fullDeconvolvedResult = []
for i in range(40, 51):
    fullDeconvolvedResult.append(tifffile.imread('Data/03_Reconstructed/%s/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39)))
# Transpose to order as z,t,y,x (images are stored as z,x,y, to match Nils's original images)
maskedObject = np.array(fullDeconvolvedResult).transpose((1,0,3,2)).copy()
maskedObject[zCropSlice] = 0

In [ ]:
# Forward-project the masked object
print(maskedObject.shape)
t1 = time.time()
unwantedCameraImages = lfdeconv.ForwardProjectACC(hMatrix, maskedObject, planes=None, progress=tqdm, logPrint=False, projector=projectorClass())
t2 = time.time()
#plf.SetThreadFileName("")
print(t2-t1)
print(unwantedCameraImages.shape)

In [ ]:
# Generate new camera images without that contribution.
# There's a risk that this will lead to negative values in the reduced camera images.
# All I can really do is limit things at a slightly non-negative value,
# but if there are too many negative values then that is a cause for concern.
# In fact, conveniently, most of the negative energy is not in the crop region
# I plan to do my initial experiments on.
reducedCameraImages = (inputImage - unwantedCameraImages)[:,342:703,361:722]
eps = 1e-6
for i in range(reducedCameraImages.shape[0]):
    negEnergy = -np.sum(reducedCameraImages[i][reducedCameraImages[i]<eps])
    posEnergy = np.sum(reducedCameraImages[i]) + negEnergy
    negFraction = negEnergy/posEnergy
    print('Image %d residual pixel range %.1f - %.1f.  Negative energy %.2f%%' 
              % (i, np.min(reducedCameraImages[i]), np.max(reducedCameraImages[i]), negFraction*100))
    reducedCameraImages[i] = np.maximum(reducedCameraImages[i], eps)

### Do the initial deconvolution of these reduced camera images
I am hoping that this will look pretty much idential to the equivalent crop of the original reconstructed volume.

- It basically does, but there are artefacts visible around the edge and also grid-corner artefacts in the centre. These are present even in the original images, but it looks like they are a bit brighter in the cropped images. 
- I might be able to suppress the edge artefacts by adjusting the normalisation of the error ratio at the edges of the volume. I should probably try that, since the edge effects may matter in these small images.
- I am also going to have to be very careful about the corner artefacts, because they are liable to confuse my flow warping.

In [ ]:
hMatrixCrop = psfmatrix.LoadMatrix('PSFmatrix/fdnormPSFMatrix_M22.2NA0.5MLPitch125fml3125from-56to56zspacing4Nnum19lambda520n1.33.mat')
print(reducedCameraImages.shape)

In [ ]:
projector = projectorClass()
projector.cacheFH = True
t1 = time.time()
Htf = lfdeconv.BackwardProjectACC(hMatrixCrop, reducedCameraImages, planes=None, progress=tqdm, logPrint=False, projector=projector)
t2 = time.time()
print(t2-t1)

In [ ]:
croppedDeconvolvedResult = lfdeconv.DeconvRL(hMatrixCrop, Htf, maxIter=maxiter, Xguess=Htf.copy(), logPrint=False, projector=projector)
print(croppedDeconvolvedResult.shape)

In [ ]:
if True:
    # Save the results
    for i in range(croppedDeconvolvedResult.shape[1]):
        tifffile.imsave('Data/03_Reconstructed/%s-cropped/Cam_%s_%d_X%d_N19.tif'%(lr, lr, 40+i, 1+i), 
                        croppedDeconvolvedResult[:,i].transpose((0,2,1)))# Transpose to match Matlab image appearance
        
    # Calculate MIPs, for inspection purposes
    for i in range(40, 51):
        im = tifffile.imread('Data/03_Reconstructed/%s-cropped/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39))
        tifffile.imsave('Data/03_Reconstructed/z-mips-%s-cropped/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39), 
                        np.max(im, axis=0))
        tifffile.imsave('Data/03_Reconstructed/y-mips-%s-cropped/Cam_%s_%d_X%d_N19.tif'%(lr, lr, i, i-39), 
                        np.max(im, axis=1))        

In [ ]:
if True:
    plf.SetThreadFileName("threads_dual_t.txt")
    t1 = time.time()
    Htf = lfdeconv.BackwardProjectACC(hMatrixCrop, reducedCameraImages[9:11], logPrint=False, projector=projectorClass())
    t2 = time.time()
    print(t2-t1)
    plf.SetThreadFileName("")

In [ ]:
print(Htf.shape)
twoTimepoints = Htf[:,9:11]
temp = lfdeconv.DeconvRL(hMatrixCrop, twoTimepoints, maxIter=maxiter, Xguess=twoTimepoints.copy(), logPrint=False, projectorClass=projectorClass)
print(temp.shape)